# NCAA ML contest - Feature engineering

In [28]:
# libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlalchemy 

## Import data

Data source: https://www.kaggle.com/c/ncaam-march-mania-2021/data

In [29]:
# read in all CSV files
datasets = os.listdir('../resources/kaggle_data/')
dfs = {}
for file in datasets:
    dfs[file[:-4]] = pd.DataFrame(pd.read_csv(f'../resources/kaggle_data/{file}', encoding='cp1252'))

In [30]:
# # preview all DFs
# df_list = list(dfs.keys())
# for x in df_list:
#     print(x)
#     print(dfs[x])
#     print('\n-----\n')

In [31]:
# create relevant dataframes
seed_df = dfs['MNCAATourneySeeds']
reg_short_df = dfs['MRegularSeasonCompactResults']
tourney_short_df = dfs['MNCAATourneyCompactResults']
reg_long_df = dfs['MRegularSeasonDetailedResults']
tourney_long_df = dfs['MNCAATourneyDetailedResults']
massey_df = dfs['MMasseyOrdinals']
teams_df = dfs['MTeams']
game_cities_df = dfs['MGameCities']
secondary_short_df = dfs['MSecondaryTourneyCompactResults']
coaches_df = dfs['MTeamCoaches']
submission_example_df = dfs['MSampleSubmissionStage1']

## Transform and clean

In [32]:
def get_stat_avg(stat, main_df):
    '''Return DF with specified stat for team and season'''
    grouped_wscore = reg_long_df.groupby(['WTeamID','Season'],as_index=False)[f'W{stat}'].sum()\
                                .rename(columns={'WTeamID':'TeamID'})
    grouped_lscore = reg_long_df.groupby(['LTeamID','Season'],as_index=False)[f'L{stat}'].sum()\
                                .rename(columns={'LTeamID':'TeamID'})
    merge_grouped = pd.merge(grouped_wscore, grouped_lscore, how='left', on=['TeamID','Season'])
    grouped_wcount = reg_long_df.groupby(['WTeamID','Season'],as_index=False)['WScore'].count()\
                                .rename(columns={'WTeamID':'TeamID','WScore':'WinCount'})
    grouped_lcount = reg_long_df.groupby(['LTeamID','Season'],as_index=False)['LScore'].count()\
                                .rename(columns={'LTeamID':'TeamID','LScore':'LoseCount'})
    merge_grouped = pd.merge(merge_grouped, grouped_wcount, how='left', on=['TeamID','Season']).fillna(0)
    merge_grouped = pd.merge(merge_grouped, grouped_lcount, how='left', on=['TeamID','Season']).fillna(0)
    merge_grouped['GameCount'] = merge_grouped['WinCount'] + merge_grouped['LoseCount']
    merge_grouped[f'Total{stat}'] = merge_grouped[f'W{stat}'] + merge_grouped[f'L{stat}']
#     merge_grouped = merge_grouped.fillna(0)
    merge_grouped[f'Avg{stat}'] = merge_grouped[f'Total{stat}']/merge_grouped['GameCount']
    
#     cols_to_use = merge_grouped.columns.difference(main_df.columns)

    merged_main = pd.merge(main_df, merge_grouped,
                           how='left',
                           left_on=['Team1','Season'],
                           right_on=['TeamID','Season'],
                           suffixes=('', ''))
    merged_main = pd.merge(merged_main, merge_grouped,
                           how='left',
                           left_on=['Team2','Season'],
                           right_on=['TeamID','Season'],
                           suffixes=('_Team1', '_Team2'))
    merged_main = merged_main.drop(columns=['TeamID_Team1','TeamID_Team2',
                                   f'W{stat}_Team1',f'W{stat}_Team2',
                                   f'L{stat}_Team1',f'L{stat}_Team2',
                                   f'Total{stat}_Team1',f'Total{stat}_Team2',])

    merged_main = merged_main.loc[:,~merged_main.columns.duplicated()]
    return merged_main

In [33]:
# make boolean columns for Tourney
reg_short_df['Tourney'] = 0
tourney_short_df['Tourney'] = 1

# append reg season and tourney DFs
combined_df = reg_short_df.append(tourney_short_df)
combined_df['ScoreDiff'] = combined_df['WScore'] - combined_df['LScore']

# create team 1 and 2 columns based on ID
combined_df['Team1'] = np.where(combined_df['WTeamID']<combined_df['LTeamID'],
                                combined_df['WTeamID'],
                                combined_df['LTeamID'])
combined_df['Team2'] = np.where(combined_df['WTeamID']>combined_df['LTeamID'],
                                combined_df['WTeamID'],
                                combined_df['LTeamID'])

combined_df = combined_df[['Season','DayNum','Team1','Team2','Tourney','WLoc','WTeamID','ScoreDiff']]

# add team 1 tourney seed column
merged_df = pd.merge(combined_df, seed_df, 
                     how='left', 
                     left_on=['Season','Team1'], 
                     right_on=['Season','TeamID'])\
            .drop(columns=['TeamID'])

merged_df = merged_df.rename(columns={'Seed':'Team1Seed'})

# add team 2 tourney seed column
merged_df2 = pd.merge(merged_df, seed_df, 
                     how='left', 
                     left_on=['Season','Team2'], 
                     right_on=['Season','TeamID'])\
            .drop(columns=['TeamID'])

merged_df2 = merged_df2.rename(columns={'Seed':'Team2Seed'})

# merged_df2['Team1Seed'] = np.where(merged_df2['Tourney']==0, 'N/A', merged_df2['Team1Seed'])
# merged_df2['Team2Seed'] = np.where(merged_df2['Tourney']==0, 'N/A', merged_df2['Team2Seed'])

# merged_df2 = merged_df2.fillna('N/A')

# add first/last D1 year
merged_df3 = pd.merge(merged_df2, teams_df, how='left', left_on='Team1', right_on='TeamID')
merged_df3 = merged_df3.drop(columns=['TeamID','TeamName'])
merged_df3 = merged_df3.rename(columns={'FirstD1Season':'Team1FirstYear','LastD1Season':'Team1LastYear'})

merged_df3 = pd.merge(merged_df3, teams_df, how='left', left_on='Team2', right_on='TeamID')
merged_df3 = merged_df3.drop(columns=['TeamID','TeamName'])
merged_df3 = merged_df3.rename(columns={'FirstD1Season':'Team2FirstYear','LastD1Season':'Team2LastYear'})

######

# winning team boolean
merged_df3['WTeam'] = np.where(merged_df2['WTeamID']==merged_df2['Team1'],1,0)

# change seed type
merged_df3['Team1Seed'] = merged_df3['Team1Seed'].str.strip().str[1:3]
merged_df3['Team1Seed'] = np.where(len(merged_df3['Team1Seed'])>2, 
                                   merged_df3['Team1Seed'].str[:2], 
                                   merged_df3['Team1Seed'])
merged_df3['Team2Seed'] = merged_df3['Team2Seed'].str.strip().str[1:3]
merged_df3['Team2Seed'] = np.where(len(merged_df3['Team2Seed'])>2, 
                                   merged_df3['Team2Seed'].str[:2], 
                                   merged_df3['Team2Seed'])

merged_df3 = merged_df3.fillna('30')

merged_df3 = merged_df3.astype({'Team1Seed':'int','Team2Seed':'int'})

#######

# add team rank averages
massey_season_avg = massey_df.groupby(['Season','TeamID'], as_index=False)['OrdinalRank'].mean()

merged_df4 = pd.merge(merged_df3, massey_season_avg, 
                       how='left', 
                       left_on=['Season','Team1'], 
                       right_on=['Season','TeamID'])

merged_df5 = pd.merge(merged_df4, massey_season_avg, 
                       how='left', 
                       left_on=['Season','Team2'], 
                       right_on=['Season','TeamID'])

merged_df5 = merged_df5.rename(columns={'OrdinalRank_x':'Team1RankMean','OrdinalRank_y':'Team2RankMean'})\
    .drop(columns=['TeamID_x','TeamID_y'])

merged_df5 = merged_df5.fillna(500)

##########

grouped_wscore = reg_short_df.groupby(['WTeamID','Season'],as_index=False)['WScore'].sum()\
                            .rename(columns={'WTeamID':'TeamID'})
grouped_lscore = reg_short_df.groupby(['LTeamID','Season'],as_index=False)[f'LScore'].sum()\
                            .rename(columns={'LTeamID':'TeamID'})
merge_grouped = pd.merge(grouped_wscore, grouped_lscore, how='left', on=['TeamID','Season'])
grouped_wcount = reg_short_df.groupby(['WTeamID','Season'],as_index=False)['WScore'].count()\
                            .rename(columns={'WTeamID':'TeamID','WScore':'WinCount'})
grouped_lcount = reg_short_df.groupby(['LTeamID','Season'],as_index=False)['LScore'].count()\
                            .rename(columns={'LTeamID':'TeamID','LScore':'LoseCount'})
merge_grouped = pd.merge(merge_grouped, grouped_wcount, how='left', on=['TeamID','Season']).fillna(0)
merge_grouped = pd.merge(merge_grouped, grouped_lcount, how='left', on=['TeamID','Season']).fillna(0)
merge_grouped['GameCount'] = merge_grouped['WinCount'] + merge_grouped['LoseCount']
merge_grouped = merge_grouped.fillna(0)
merge_grouped[f'TotalScore'] = merge_grouped[f'WScore'] + merge_grouped[f'LScore']
#     merge_grouped = merge_grouped.fillna(0)
merge_grouped[f'AvgScore'] = merge_grouped[f'TotalScore']/merge_grouped['GameCount']

# # add win% columns
merge_grouped['Win%'] = ((merge_grouped['WinCount']/merge_grouped['GameCount'])*10)**2


# Merge with main DF
merged_df6 = pd.merge(merged_df5, merge_grouped, 
                      how='left', 
                      left_on=['Team1','Season'], 
                      right_on=['TeamID','Season'], 
                      suffixes=['_Team1','_Team2'])
merged_df6 = pd.merge(merged_df6, merge_grouped, 
                      how='left', 
                      left_on=['Team2','Season'], 
                      right_on=['TeamID','Season'],
                      suffixes=['_Team1','_Team2'])

merged_df6 = merged_df6.drop(columns=['LoseCount_Team1','LoseCount_Team2',
                                      'TotalScore_Team1','TotalScore_Team2',
                                      'TeamID_Team1','WScore_Team1','LScore_Team1',
                                      'TeamID_Team2','WScore_Team2','LScore_Team2'])

merged_df6.head()

,Season,DayNum,Team1,Team2,Tourney,WLoc,WTeamID,ScoreDiff,Team1Seed,Team2Seed,...,Team1RankMean,Team2RankMean,WinCount_Team1,GameCount_Team1,AvgScore_Team1,Win%_Team1,WinCount_Team2,GameCount_Team2,AvgScore_Team2,Win%_Team2
0,1985,20,1228,1328,0,N,1228,17,3,1,...,500.0,500.0,23.0,31.0,68.225806,55.046826,25.0,30.0,89.833333,69.444444
1,1985,25,1106,1354,0,H,1106,7,30,30,...,500.0,500.0,10.0,24.0,71.625000,17.361111,9.0,24.0,68.208333,14.062500
2,1985,25,1112,1223,0,H,1112,7,10,30,...,500.0,500.0,18.0,27.0,66.518519,44.444444,17.0,25.0,68.320000,46.240000
3,1985,25,1165,1432,0,H,1165,16,30,30,...,500.0,500.0,12.0,24.0,61.375000,25.000000,11.0,23.0,63.478261,22.873346
4,1985,25,1192,1447,0,H,1192,12,16,30,...,500.0,500.0,19.0,28.0,67.892857,46.045918,8.0,24.0,73.333333,11.111111


## Add/drop stats

In [34]:
stat_cols = reg_long_df.columns[8:]

stat_list = [x[1:] for x in stat_cols]
for stat in stat_list:
    merged_df6 = get_stat_avg(stat, merged_df6)
    
merged_df6

,Season,DayNum,Team1,Team2,Tourney,WLoc,WTeamID,ScoreDiff,Team1Seed,Team2Seed,...,AvgAst_Team1,AvgAst_Team2,AvgTO_Team1,AvgTO_Team2,AvgStl_Team1,AvgStl_Team2,AvgBlk_Team1,AvgBlk_Team2,AvgPF_Team1,AvgPF_Team2
0,1985,20,1228,1328,0,N,1228,17,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1985,25,1106,1354,0,H,1106,7,30,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1985,25,1112,1223,0,H,1112,7,10,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1985,25,1165,1432,0,H,1165,16,30,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1985,25,1192,1447,0,H,1192,12,16,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169126,2019,146,1120,1246,1,N,1120,6,5,2,...,14.411765,13.848485,12.147059,12.636364,9.294118,6.030303,4.764706,5.060606,18.382353,16.303030
169127,2019,146,1181,1277,1,N,1277,1,1,2,...,15.911765,18.941176,13.117647,12.852941,9.470588,5.235294,6.823529,5.470588,15.852941,16.911765
169128,2019,152,1277,1403,1,N,1403,10,2,3,...,18.941176,14.031250,12.852941,12.375000,5.235294,7.375000,5.470588,4.906250,16.911765,17.781250
169129,2019,152,1120,1438,1,N,1438,1,5,1,...,14.411765,14.500000,12.147059,9.031250,9.294118,5.625000,4.764706,3.812500,18.382353,14.656250


In [35]:
# add shooting %
merged_df6['FG%_Team1'] = merged_df6['AvgFGM_Team1']/merged_df6['AvgFGA_Team1']
merged_df6['FG%_Team2'] = merged_df6['AvgFGM_Team2']/merged_df6['AvgFGA_Team2']
merged_df6['FG3%_Team1'] = merged_df6['AvgFGM3_Team1']/merged_df6['AvgFGA3_Team1']
merged_df6['FG3%_Team2'] = merged_df6['AvgFGM3_Team2']/merged_df6['AvgFGA3_Team2']
merged_df6.columns

Index(['Season', 'DayNum', 'Team1', 'Team2', 'Tourney', 'WLoc', 'WTeamID',
       'ScoreDiff', 'Team1Seed', 'Team2Seed', 'Team1FirstYear',
       'Team1LastYear', 'Team2FirstYear', 'Team2LastYear', 'WTeam',
       'Team1RankMean', 'Team2RankMean', 'WinCount_Team1', 'GameCount_Team1',
       'AvgScore_Team1', 'Win%_Team1', 'WinCount_Team2', 'GameCount_Team2',
       'AvgScore_Team2', 'Win%_Team2', 'LoseCount_Team1', 'AvgFGM_Team1',
       'LoseCount_Team2', 'AvgFGM_Team2', 'AvgFGA_Team1', 'AvgFGA_Team2',
       'AvgFGM3_Team1', 'AvgFGM3_Team2', 'AvgFGA3_Team1', 'AvgFGA3_Team2',
       'AvgFTM_Team1', 'AvgFTM_Team2', 'AvgFTA_Team1', 'AvgFTA_Team2',
       'AvgOR_Team1', 'AvgOR_Team2', 'AvgDR_Team1', 'AvgDR_Team2',
       'AvgAst_Team1', 'AvgAst_Team2', 'AvgTO_Team1', 'AvgTO_Team2',
       'AvgStl_Team1', 'AvgStl_Team2', 'AvgBlk_Team1', 'AvgBlk_Team2',
       'AvgPF_Team1', 'AvgPF_Team2', 'FG%_Team1', 'FG%_Team2', 'FG3%_Team1',
       'FG3%_Team2'],
      dtype='object')

In [36]:
# remove Tourney 2015-19 records
unwanted_rows = merged_df6.loc[(merged_df6.Season > 2014) & (merged_df6.Tourney==1)]
merged_df7 = merged_df6.drop(index=unwanted_rows.index)

# convert WLoc
merged_df7['WLoc'] = np.where(merged_df7['WLoc']=='H', '1', 
         (np.where(merged_df7['WLoc']=='A', '-1', '0'))).astype('int')

merged_df7

,Season,DayNum,Team1,Team2,Tourney,WLoc,WTeamID,ScoreDiff,Team1Seed,Team2Seed,...,AvgStl_Team1,AvgStl_Team2,AvgBlk_Team1,AvgBlk_Team2,AvgPF_Team1,AvgPF_Team2,FG%_Team1,FG%_Team2,FG3%_Team1,FG3%_Team2
0,1985,20,1228,1328,0,0,1228,17,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1985,25,1106,1354,0,1,1106,7,30,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1985,25,1112,1223,0,1,1112,7,10,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1985,25,1165,1432,0,1,1165,16,30,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1985,25,1192,1447,0,1,1192,12,16,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168791,2014,146,1163,1277,1,0,1163,6,7,4,...,6.970588,6.941176,6.205882,4.823529,18.264706,18.882353,0.448350,0.474654,0.387200,0.391429
168792,2014,146,1246,1276,1,0,1246,3,8,2,...,4.764706,5.121212,6.235294,2.515152,19.000000,14.545455,0.451239,0.475788,0.325188,0.393812
168793,2014,152,1163,1196,1,0,1163,10,7,1,...,6.970588,7.205882,6.205882,2.852941,18.264706,16.205882,0.448350,0.462096,0.387200,0.368088
168794,2014,152,1246,1458,1,0,1246,1,8,2,...,4.764706,5.000000,6.235294,3.515152,19.000000,14.939394,0.451239,0.457880,0.325188,0.373178


In [37]:
merged_df7.to_csv('../resources/transformed_data/all_stats.csv', index=False)

## Feature selection and engineering

In [38]:
# # set features df
# features_df = merged_df7.drop(columns=['WTeamID','DayNum','ScoreDiff',
#                                        'Team1FirstYear','Team2FirstYear',
#                                        'Team1LastYear','Team2LastYear', 
#                                        'WinCount_Team1', 'WinCount_Team2',
#                                        'GameCount_Team1','GameCount_Team2',
#                                        'AvgScore_Team1','AvgScore_Team2'
#                                       ])  

# # remove Tourney 2015-19 records
# features_df = pd.get_dummies(features_df.loc[(features_df.Season < 2014) & (features_df.Tourney==0)])

# features_df = features_df.dropna(how='any')

# features_df = features_df.astype({
#                                   'Season':'str',
#                                   'Team1':'str',
#                                   'Team2':'str',
# #                                   'Team1FirstYear':'str',
# #                                   'Team2FirstYear':'str',
#                                  })

# features_df.shape

In [39]:
# # select features and target
# target = features_df.pop('WTeam')
# selected_features = features_df

# print(selected_features.shape)
# print(target.shape)

In [40]:
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.compose import make_column_transformer
# from sklearn.compose import make_column_selector

# ct = make_column_transformer(
#     (StandardScaler(), make_column_selector(dtype_include=np.number)),
#     (OneHotEncoder(handle_unknown='ignore'), make_column_selector(dtype_include=object))
# )

# ct.fit_transform(selected_features)

## MLP Classifier testing

In [41]:
# from sklearn.neural_network import MLPClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import make_pipeline
# from sklearn.model_selection import GridSearchCV, LeaveOneOut
# from sklearn import metrics

# os.environ['KMP_DUPLICATE_LIB_OK']='True'

# mlc = MLPClassifier(activation = 'relu', random_state=1, nesterovs_momentum=True)
# loo = LeaveOneOut()
# pipe = make_pipeline(ct, mlc)

# params = {
#           "mlpclassifier__hidden_layer_sizes":[(168,),(126,),(498,),(166,)],
#           "mlpclassifier__solver" : ('sgd','adam'), 
#           "mlpclassifier__alpha": [0.001,0.0001],
#           "mlpclassifier__learning_rate_init":[0.005,0.001]
#          }

# clf = GridSearchCV(pipe, params, cv=loo, verbose=2)

# X_train, X_test, y_train, y_test = train_test_split(selected_features, 
#                                                     target, stratify=target, 
#                                                     random_state=42)

# # clf = make_pipeline(ct, MLPClassifier(activation='relu',
# #                                       hidden_layer_sizes=100, 
# #                                       solver='sgd', 
# #                                       alpha=0.0001,
# #                                       batch_size='auto',
# #                                       learning_rate='adaptive',
# #                                       learning_rate_init=0.001,
# #                                       power_t=0.5,
# #                                       max_iter=300, 
# #                                       shuffle=True,
# #                                       random_state=69,
# #                                       tol=1e-4,
# #                                       verbose=True))

# clf.fit(X_train, y_train)

# model = clf.best_estimator_
# print("the best model and parameters are the following: {} ".format(model))

# # clf.predict_proba(X_test)
# # preds = clf.predict(X_test)
# # train_score = clf.score(X_train, y_train)
# # test_score = clf.score(X_test, y_test)
# # loss_score = metrics.log_loss(y_test, preds)*.1

# # print(f'Train score: {test_score}')
# # print(f'Test score: {test_score}')
# # print(f'Log loss: {loss_score}')

Best log loss: .8338768513668983

## Format and submission

In [43]:
submit_data = pd.DataFrame(submission_example_df['ID'].str.split('_',expand=True))
submit_data.columns = ['Season','Team1','Team2']
submit_data = submit_data.astype('int')

# make boolean columns for Tourney
submit_data['Tourney'] = 1
submit_data['WLoc'] = 'N'

# convert WLoc
submit_data['WLoc'] = np.where(submit_data['WLoc']=='H', '1', 
         (np.where(submit_data['WLoc']=='A', '-1', '0'))).astype('int')

# add team 1 and 2 tourney seed column
submit_data2 = pd.merge(submit_data, seed_df, 
                     how='left', 
                     left_on=['Season','Team1'], 
                     right_on=['Season','TeamID'])\
            .drop(columns=['TeamID'])

submit_data2 = submit_data2.rename(columns={'Seed':'Team1Seed'})

submit_data3 = pd.merge(submit_data2, seed_df, 
                     how='left', 
                     left_on=['Season','Team2'], 
                     right_on=['Season','TeamID'])\
            .drop(columns=['TeamID'])

submit_data4 = submit_data3.rename(columns={'Seed':'Team2Seed'})

# change seed data type and remove alpha characters
submit_data4['Team1Seed'] = submit_data4['Team1Seed'].str.strip().str[1:3]
submit_data4['Team1Seed'] = np.where(len(submit_data4['Team1Seed'])>2, 
                                   submit_data4['Team1Seed'].str[:2], 
                                   submit_data4['Team1Seed'])
submit_data4['Team2Seed'] = submit_data4['Team2Seed'].str.strip().str[1:3]
submit_data4['Team2Seed'] = np.where(len(submit_data4['Team2Seed'])>2, 
                                   submit_data4['Team2Seed'].str[:2], 
                                   submit_data4['Team2Seed'])

submit_data4 = submit_data4.astype({'Team1Seed':'int','Team2Seed':'int'})

# add rank data
massey_season_avg = massey_df.groupby(['Season','TeamID'], as_index=False)['OrdinalRank'].mean()

submit_data4 = pd.merge(submit_data4, massey_season_avg, 
                       how='left', 
                       left_on=['Season','Team1'], 
                       right_on=['Season','TeamID'])

submit_data5 = pd.merge(submit_data4, massey_season_avg, 
                       how='left', 
                       left_on=['Season','Team2'], 
                       right_on=['Season','TeamID'])

submit_data5 = submit_data5.rename(columns={'OrdinalRank_x':'Team1RankMean',
                                            'OrdinalRank_y':'Team2RankMean'})\
                            .drop(columns=['TeamID_x','TeamID_y'])

# test_data5 = test_data5.fillna(500)

# add first/last D1 year
submit_data5 = pd.merge(submit_data5, teams_df, how='left', left_on='Team1', right_on='TeamID')
submit_data5 = submit_data5.drop(columns=['TeamID','TeamName'])
submit_data5 = submit_data5.rename(columns={'FirstD1Season':'Team1FirstYear',
                                            'LastD1Season':'Team1LastYear'})

submit_data5 = pd.merge(submit_data5, teams_df, how='left', left_on='Team2', right_on='TeamID')
submit_data5 = submit_data5.drop(columns=['TeamID','TeamName'])
submit_data5 = submit_data5.rename(columns={'FirstD1Season':'Team2FirstYear',
                                            'LastD1Season':'Team2LastYear'})

##########

grouped_wscore = reg_short_df.groupby(['WTeamID','Season'],as_index=False)['WScore'].sum()\
                            .rename(columns={'WTeamID':'TeamID'})
grouped_lscore = reg_short_df.groupby(['LTeamID','Season'],as_index=False)[f'LScore'].sum()\
                            .rename(columns={'LTeamID':'TeamID'})
merge_grouped = pd.merge(grouped_wscore, grouped_lscore, how='left', on=['TeamID','Season'])
grouped_wcount = reg_short_df.groupby(['WTeamID','Season'],as_index=False)['WScore'].count()\
                            .rename(columns={'WTeamID':'TeamID','WScore':'WinCount'})
grouped_lcount = reg_short_df.groupby(['LTeamID','Season'],as_index=False)['LScore'].count()\
                            .rename(columns={'LTeamID':'TeamID','LScore':'LoseCount'})
merge_grouped = pd.merge(merge_grouped, grouped_wcount, how='left', on=['TeamID','Season']).fillna(0)
merge_grouped = pd.merge(merge_grouped, grouped_lcount, how='left', on=['TeamID','Season']).fillna(0)
merge_grouped['GameCount'] = merge_grouped['WinCount'] + merge_grouped['LoseCount']
merge_grouped = merge_grouped.fillna(0)
merge_grouped[f'TotalScore'] = merge_grouped[f'WScore'] + merge_grouped[f'LScore']
#     merge_grouped = merge_grouped.fillna(0)
merge_grouped[f'AvgScore'] = merge_grouped[f'TotalScore']/merge_grouped['GameCount']

# # add win% columns
merge_grouped['Win%'] = merge_grouped['WinCount']/merge_grouped['GameCount']


# Merge with main DF
submit_data6 = pd.merge(submit_data5, merge_grouped, 
                      how='left', 
                      left_on=['Team1','Season'], 
                      right_on=['TeamID','Season'], 
                      suffixes=['_Team1','_Team2'])
submit_data6 = pd.merge(submit_data6, merge_grouped, 
                      how='left', 
                      left_on=['Team2','Season'], 
                      right_on=['TeamID','Season'],
                      suffixes=['_Team1','_Team2'])

submit_data6 = submit_data6.drop(columns=['LoseCount_Team1','LoseCount_Team2',
                                      'TotalScore_Team1','TotalScore_Team2',
                                      'TeamID_Team1','WScore_Team1','LScore_Team1',
                                      'TeamID_Team2','WScore_Team2','LScore_Team2'])

In [47]:
stat_cols = reg_long_df.columns[8:]
stat_list = [x[1:] for x in stat_cols]
for stat in stat_list:
    submit_data6 = get_stat_avg(stat, submit_data6)
    
submit_data6

,Season,Team1,Team2,Tourney,WLoc,Team1Seed,Team2Seed,Team1RankMean,Team2RankMean,Team1FirstYear,...,AvgAst_Team1,AvgAst_Team2,AvgTO_Team1,AvgTO_Team2,AvgStl_Team1,AvgStl_Team2,AvgBlk_Team1,AvgBlk_Team2,AvgPF_Team1,AvgPF_Team2
0,2015,1107,1112,1,0,14,2,154.890566,7.155889,2000,...,10.437500,14.205882,11.687500,11.205882,5.656250,7.176471,1.625000,3.588235,16.500000,17.911765
1,2015,1107,1116,1,0,14,5,154.890566,26.339623,2000,...,10.437500,16.147059,11.687500,11.735294,5.656250,7.764706,1.625000,4.764706,16.500000,18.970588
2,2015,1107,1124,1,0,14,3,154.890566,17.802834,2000,...,10.437500,14.406250,11.687500,12.375000,5.656250,8.031250,1.625000,3.843750,16.500000,16.718750
3,2015,1107,1125,1,0,14,15,154.890566,132.294340,2000,...,10.437500,15.193548,11.687500,13.741935,5.656250,6.354839,1.625000,1.967742,16.500000,16.451613
4,2015,1107,1129,1,0,14,11,154.890566,63.752179,2000,...,10.437500,11.709677,11.687500,10.225806,5.656250,5.967742,1.625000,2.322581,16.500000,17.225806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11385,2019,1449,1459,1,0,9,7,43.404278,48.756124,1985,...,11.676471,14.700000,13.352941,11.233333,9.000000,6.133333,5.735294,2.933333,18.411765,17.500000
11386,2019,1449,1463,1,0,9,14,43.404278,91.541037,1985,...,11.676471,17.000000,13.352941,13.250000,9.000000,5.750000,5.735294,4.321429,18.411765,17.000000
11387,2019,1458,1459,1,0,5,7,19.631308,48.756124,1985,...,12.757576,14.700000,9.545455,11.233333,5.121212,6.133333,4.212121,2.933333,15.060606,17.500000
11388,2019,1458,1463,1,0,5,14,19.631308,91.541037,1985,...,12.757576,17.000000,9.545455,13.250000,5.121212,5.750000,4.212121,4.321429,15.060606,17.000000


In [49]:
# add shooting %
submit_data6['FG%_Team1'] = submit_data6['AvgFGM_Team1']/submit_data6['AvgFGA_Team1']
submit_data6['FG%_Team2'] = submit_data6['AvgFGM_Team2']/submit_data6['AvgFGA_Team2']
submit_data6['FG3%_Team1'] = submit_data6['AvgFGM3_Team1']/submit_data6['AvgFGA3_Team1']
submit_data6['FG3%_Team2'] = submit_data6['AvgFGM3_Team2']/submit_data6['AvgFGA3_Team2']

submit_data6.columns

Index(['Season', 'Team1', 'Team2', 'Tourney', 'WLoc', 'Team1Seed', 'Team2Seed',
       'Team1RankMean', 'Team2RankMean', 'Team1FirstYear', 'Team1LastYear',
       'Team2FirstYear', 'Team2LastYear', 'WinCount_Team1', 'GameCount_Team1',
       'AvgScore_Team1', 'Win%_Team1', 'WinCount_Team2', 'GameCount_Team2',
       'AvgScore_Team2', 'Win%_Team2', 'LoseCount_Team1', 'AvgFGM_Team1',
       'LoseCount_Team2', 'AvgFGM_Team2', 'AvgFGA_Team1', 'AvgFGA_Team2',
       'AvgFGM3_Team1', 'AvgFGM3_Team2', 'AvgFGA3_Team1', 'AvgFGA3_Team2',
       'AvgFTM_Team1', 'AvgFTM_Team2', 'AvgFTA_Team1', 'AvgFTA_Team2',
       'AvgOR_Team1', 'AvgOR_Team2', 'AvgDR_Team1', 'AvgDR_Team2',
       'AvgAst_Team1', 'AvgAst_Team2', 'AvgTO_Team1', 'AvgTO_Team2',
       'AvgStl_Team1', 'AvgStl_Team2', 'AvgBlk_Team1', 'AvgBlk_Team2',
       'AvgPF_Team1', 'AvgPF_Team2', 'FG%_Team1', 'FG%_Team2', 'FG3%_Team1',
       'FG3%_Team2'],
      dtype='object')

In [51]:
col_list = merged_df6.columns.tolist()
remove_list = ['DayNum','WTeam','WTeamID','ScoreDiff']
for x in remove_list:
    col_list.remove(x)
submit_data6 = submit_data6[col_list]
submit_data6.columns

Index(['Season', 'Team1', 'Team2', 'Tourney', 'WLoc', 'Team1Seed', 'Team2Seed',
       'Team1FirstYear', 'Team1LastYear', 'Team2FirstYear', 'Team2LastYear',
       'Team1RankMean', 'Team2RankMean', 'WinCount_Team1', 'GameCount_Team1',
       'AvgScore_Team1', 'Win%_Team1', 'WinCount_Team2', 'GameCount_Team2',
       'AvgScore_Team2', 'Win%_Team2', 'LoseCount_Team1', 'AvgFGM_Team1',
       'LoseCount_Team2', 'AvgFGM_Team2', 'AvgFGA_Team1', 'AvgFGA_Team2',
       'AvgFGM3_Team1', 'AvgFGM3_Team2', 'AvgFGA3_Team1', 'AvgFGA3_Team2',
       'AvgFTM_Team1', 'AvgFTM_Team2', 'AvgFTA_Team1', 'AvgFTA_Team2',
       'AvgOR_Team1', 'AvgOR_Team2', 'AvgDR_Team1', 'AvgDR_Team2',
       'AvgAst_Team1', 'AvgAst_Team2', 'AvgTO_Team1', 'AvgTO_Team2',
       'AvgStl_Team1', 'AvgStl_Team2', 'AvgBlk_Team1', 'AvgBlk_Team2',
       'AvgPF_Team1', 'AvgPF_Team2', 'FG%_Team1', 'FG%_Team2', 'FG3%_Team1',
       'FG3%_Team2'],
      dtype='object')

In [52]:
submit_data7.to_csv('../resources/transformed_data/all_stats_submit.csv', index=False)